In [63]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

In [64]:
# Mở trình duyệt Google Chrome
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito') # Tab ẩn danh
options.add_argument("--start-maximized") # Full window
# options.add_argument('headless') # Không hiển thị chrome
s = Service('../chromedriver.exe')

In [65]:
# Tìm url sp theo keyword
# keyword = 'Tai nghe Bluetooh'

# url = 'https://tiki.vn/search?q=%s'%('%20'.join(keyword.split(' ')))
# driver.get(url)
# sleep(1)

# element= WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="CatalogProducts__Wrapper-sc-1r8ct7c-0 fKlIZg"]')))
# html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
# soup = BeautifulSoup(html_of_interest, 'lxml')

# raw_links = soup.select('a[data-view-id="product_list_item"]')
# list_link = []

# for num_link in range(len(raw_links)):
#     link = raw_links[num_link].attrs['href']

#     if 'tka.tiki.vn' in link:
#         list_link.append('https:' + link)
#     else:
#         list_link.append('https://tiki.vn' + link)

In [66]:
def get_url(driver, url): # Mở url
    driver.get(url)

    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main')))

    total_height = int(driver.execute_script("return document.body.scrollHeight"))

    for i in range(1, total_height, 5):
        driver.execute_script("window.scrollTo(0, {});".format(i))

    check_total_height = int(driver.execute_script("return document.body.scrollHeight"))
    if check_total_height > total_height:
        for i in range(total_height, check_total_height, 5):
            driver.execute_script("window.scrollTo(0, {});".format(i))

In [67]:
def get_soup(driver, url=None): # Trả về soup
    if url != None:
        get_url(driver, url)

    element= WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main > div[class="ContainerRevamp-sc-1hvvgwz-0 dOGdaN"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')

    return soup

In [68]:
def get_product_info(soup): # Trả về 1 dict thông tin sản phẩm
    dict_product_info = {}

    # Tên sản phẩm
    dict_product_info['Name'] = soup.select('h1[class="Title__TitledStyled-sc-1kxsq5b-0 cvyKhs"]')[0].text

    # Thương hiệu
    dict_product_info['Brand'] = soup.select('a[data-view-id="pdp_details_view_brand"]')[0].text

    # Số lượng bán
    dict_product_info['Quantity'] = soup.select('div[data-view-id="pdp_quantity_sold"]')[0].text.split()[-1]

    # Số lượt đánh giá
    dict_product_info['Reviews count'] = soup.select('a[data-view-id="pdp_main_view_review"]')[0].text.strip('()')

    # Đánh giá sao
    dict_product_info['Star rating'] = soup.select('div[class="styles__StyledReview-sc-1swui9f-1 dXPbue"] > div')[0].text

    # Giá bán
    dict_product_info['Price'] = soup.select('div[class="product-price__current-price"]')[0].text.replace('₫', '')

    # Thông số lựa chọn (Màu sắc, dung lượng,...)
    list_option_label = soup.select('div[class="styles__ProductOptionsWrapper-sc-18rzur4-0 jZCObm"] > div')
    for option_label in list_option_label:
        list_option = option_label.select('div[class="styles__OptionListWrapper-sc-1pikfxx-2 jgDdBJ"] > div')
        list_all_option = []

        for option in list_option:
            list_all_option.append(option.text)

        dict_product_info[option_label.attrs['data-view-label']] = ', '.join(list_all_option)

    # Tên shop
    dict_product_info['Shop name'] = soup.select('span[class="seller-name"]')[0].text

    # Đánh giá sao shop
    dict_product_info['Shop rating'] = soup.select('div[class="item review"] > div')[0].text

    # Số lượt đánh giá shop
    dict_product_info['Shop rating count'] = soup.select('div[class="item review"] > div')[1].text.strip('()đánh giá')

    # Thông tin chi tiết
    product_info_soup = soup.select('div[class="WidgetTitle__WidgetContainerStyled-sc-1ikmn8z-0 iHMNqO"]')[-5]
    product_info = product_info_soup.select('div > div > div > span')

    for i in range(0, len(product_info), 2):
        # product_info[i].text: nhãn thông tin
        # product_info[i+1].text: thông tin
        dict_product_info[product_info[i].text] = product_info[i+1].text

    # Mô tả sản phẩm
    dict_product_info['Describe'] = soup.select('div[class="WidgetTitle__WidgetContainerStyled-sc-1ikmn8z-0 iHMNqO"]')[-4].text

    return dict_product_info

In [69]:
def get_review(driver, soup): # Trả về 1 list
    all_reviews = []

    while True:
        # Lấy thông tin review
        list_review = soup.select('div[class="style__StyledComment-sc-hup0ux-5 jSHEVq review-comment"]')

        for review in list_review:
            dict_review = {}

            # Tên
            dict_review['Reviewer name'] = review.select('div[class="review-comment__user-name"]')[0].text

            # Nội dung
            dict_review['Content'] = review.select('div[class="review-comment__content"]')[0].text

            # Đánh giá
            dict_review['Rating'] = review.select('div[class="review-comment__title"]')[0].text

            # Thời gian đánh giá
            crawl_time = datetime.today().strftime('%Y-%m-%d')
            dict_review['Rating date'] = review.select('div[class="review-comment__created-date"] > span')[0].text + '<>(%s)'%crawl_time

            all_reviews.append(dict_review)
        
        # Check có phải page cuối
        btn_color = driver.find_element(By.CSS_SELECTOR, 'a[class="btn next"] > svg').get_attribute('color')
        if btn_color == '#C4C4CF':
            break

        # Bấm chuyển page
        driver.find_element(By.CSS_SELECTOR, 'a[class="btn next"]').click()
        sleep(5)

        # Lấy soup mới
        soup = get_soup(driver)

    return all_reviews

In [70]:
# Test (Mạng ổn định: Sản phẩm tầm 50 review thường mất 1 phút để hoàn thành)
driver = webdriver.Chrome(service=s, options=options)
product_url = 'https://tiki.vn/apple-iphone-15-p271966786.html?spid=271966788'

product_soup = get_soup(driver, url=product_url)
product_info = get_product_info(product_soup)
product_reviews = get_review(driver, product_soup)